This is the third part of the IBM Capstone Project, where we'll be exploring vegan restaurant prospects in the downtown Toronto region. The purpose of this exploration will allow users to determine the saturation levels of quality vegan restaurants in a particular area and look for gaps within the neighbourhoods.

Problem & Background: 

It is extremely common to see vegan restaurants in mainstream cities like Toronto, Ontario. Due to the wide adoption of veganism, there is a huge market for the vegan restaurant industry.

However, one of the major issue for this unique cuisine in the potential for over-saturation of these restaurants, increasing competition and reducing market share. If a new business owner wants to open a vegan restaurant where should they set up shop? This is an important question and one that can be explored using Toronto data and Foursquare API. Foursquare will allow us to explore current quality vegan restaurants in the downtown area, show ratings and indicate where potential market needs are.  


Data Description: 

The data that will be used for this project is from this wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Using lxml parsing techniques, information about the neighbourhood, borough and postal code will be scraped and utilized to assist in the exploration of common vegan restaurants venues in the Toronto neighbourhoods. Wikipedia has pulled the respective postal codes from the Canada Post website. 


Firstly, the data regarding Toronto Neighbourhoods, Boroughs and Postal Code will be scraped from the Wikipedia page. This information will then be added to a new pandas data frame containing three column headings "Postcode, Boroughs, Neighbourhoods".  

A Postcode is the postal code of the particular Neighbourhood and will assist in dividing the Neighbourhoods into the appropriate Boroughs.

A Borough is considered an administrative division of a municipality.

A Neighbourhood is a geographically localized community within a municipality. 


Methedology: Import Necessary Libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import requests
from lxml import html #parsing python package

Store the Wikipedia Page and utilize doc path to read into the html tags

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = html.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



Create a for loop to iterate over each cell of the table

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
       # Check if row does not have an assigned borough
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
     

Create Pandas DataFrame

In [6]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [7]:
#ensure all data are objects
df.dtypes

Postcode           object
Borough            object
Neighbourhood\n    object
dtype: object

In [8]:
#reset the index
df.set_index('Borough')

,Postcode,Neighbourhood
Borough,,
Not assigned,M1A,Not assigned\n
Not assigned,M2A,Not assigned\n
North York,M3A,Parkwoods\n
North York,M4A,Victoria Village\n
Downtown Toronto,M5A,Harbourfront\n
Downtown Toronto,M5A,Regent Park\n
North York,M6A,Lawrence Heights\n
North York,M6A,Lawrence Manor\n
Queen's Park,M7A,Not assigned\n


In [9]:
#isolate rows without Non assigned in Borough column
df = df[df.Borough != 'Not assigned']


In [10]:
# replace the trailling \n with space
df = df.replace('\n','', regex=True)

In [11]:
#replace not assigned value in neighbourhood column with value from Borough column
df.loc[df['Neighbourhood\n'] == 'Not assigned','Neighbourhood\n'] = df['Borough']

In [12]:
#sanity check
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [13]:
# eliminate duplicate postals to combine multiple neighbourhoods into a single borough and postal
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood\n'].apply(', '.join).reset_index()

In [14]:
df.shape

(103, 3)

Had issues using the Geocoder package decided to combine the dataframes instead

In [15]:
data = pd.read_csv('geo.csv')

In [16]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
bigdata = pd.concat([data, df], axis=1)

In [18]:
bigdata.drop(['Postal Code'], axis=1)

,Latitude,Longitude,Postcode,Borough,Neighbourhood
0,43.806686,-79.194353,M1B,Scarborough,"Rouge, Malvern"
1,43.784535,-79.160497,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,43.763573,-79.188711,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,43.770992,-79.216917,M1G,Scarborough,Woburn
4,43.773136,-79.239476,M1H,Scarborough,Cedarbrae
5,43.744734,-79.239476,M1J,Scarborough,Scarborough Village
6,43.727929,-79.262029,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,43.711112,-79.284577,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,43.716316,-79.239476,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,43.692657,-79.264848,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [19]:
bigdata = bigdata[['Postcode', 'Borough', 'Neighbourhood\n', 'Latitude', 'Longitude']] 

In [20]:
bigdata

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Secondly lets exxplore the current neighbourhoods Via Map

Obtain Foursquare credentials

In [21]:
#obtian foursquare credentials
CLIENT_ID = 'FUDKBWXJDURUZREEPDPNVFOKNV1TUXP143R33V1XVFUN5HKO'# your Foursquare ID
CLIENT_SECRET = 'YK5GF23QXFMMYV54XZC4PETX4ZAE0YA1WSCZILUQM3WLN1EE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FUDKBWXJDURUZREEPDPNVFOKNV1TUXP143R33V1XVFUN5HKO
CLIENT_SECRET:YK5GF23QXFMMYV54XZC4PETX4ZAE0YA1WSCZILUQM3WLN1EE


In [22]:
bigdata = bigdata[bigdata['Borough'].str.contains("Toronto")]


In [23]:
#filter out map of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bigdata['Latitude'], bigdata['Longitude'], bigdata['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's narrow it down even more to the Downtown Toronto

In [25]:
downtown_data = bigdata[bigdata['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [26]:
downtown_data= downtown_data.set_index("Neighbourhood\n")

In [27]:
#remove this neighbourhood as pretty out of scope
downtown_data= downtown_data.drop("Rosedale", axis=0)

In [28]:
#remove this neighbourhood as pretty out of scope
downtown_data= downtown_data.drop("Christie", axis=0)

In [29]:
downtown_data= downtown_data.reset_index()

In [30]:
downtown_data

,Neighbourhood,Postcode,Borough,Latitude,Longitude
0,"Cabbagetown, St. James Town",M4X,Downtown Toronto,43.667967,-79.367675
1,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160
2,"Harbourfront, Regent Park",M5A,Downtown Toronto,43.654260,-79.360636
3,"Ryerson, Garden District",M5B,Downtown Toronto,43.657162,-79.378937
4,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
5,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
6,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383
7,"Adelaide, King, Richmond",M5H,Downtown Toronto,43.650571,-79.384568
8,"Harbourfront East, Toronto Islands, Union Station",M5J,Downtown Toronto,43.640816,-79.381752
9,"Design Exchange, Toronto Dominion Centre",M5K,Downtown Toronto,43.647177,-79.381576


In [31]:
#map of downtown Toronto (filtered)
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [32]:
# create map of Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [33]:
downtown_data.loc[12, 'Neighbourhood\n']

'Chinatown, Grange Park, Kensington Market'

In [34]:
neighborhood_latitude = downtown_data.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[12, 'Longitude'] # neighborhood longitude value

neighborhood_name= downtown_data.loc[12, 'Neighbourhood\n'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Chinatown, Grange Park, Kensington Market are 43.6532057, -79.4000493.


In [35]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 700 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=FUDKBWXJDURUZREEPDPNVFOKNV1TUXP143R33V1XVFUN5HKO&client_secret=YK5GF23QXFMMYV54XZC4PETX4ZAE0YA1WSCZILUQM3WLN1EE&v=20180604&ll=43.6532057,-79.4000493&radius=700&limit=100'

Third, make a GET call to Foursquare API to receive the top rated restuarants for one specific neighbourhood.

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb16377351e3d3ef10e7e0d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kensington',
  'headerFullLocation': 'Kensington, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 43.65950570630001,
    'lng': -79.39135825277472},
   'sw': {'lat': 43.64690569369999, 'lng': -79.40874034722529}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50427a03e4b08d9f5931f593',
       'name': 'Seven Lives - Tacos y Mariscos',
       'location': {'address': '69 Kensington Ave',
        'crossStreet': 'Baldwin St',
        'lat': 43.65441817160925,
        'lng': -79.40054467846261,
        'labe

In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Seven Lives - Tacos y Mariscos,Mexican Restaurant,43.654418,-79.400545
1,Kid Icarus,Arts & Crafts Store,43.653933,-79.401719
2,The Moonbean Cafe,Café,43.654147,-79.400182
3,Essence of Life Organics,Organic Grocery,43.654111,-79.400431
4,Little Pebbles,Coffee Shop,43.654883,-79.400264


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Create function to allow for iteration of the other neighbourhoods in the Downtown borough

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:

nearby_venues = getNearbyVenues(names=downtown_data['Neighbourhood\n'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city


In [42]:
print(nearby_venues.shape)
nearby_venues.head()

(1266, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
1,"Cabbagetown, St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
2,"Cabbagetown, St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
3,"Cabbagetown, St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant


In [43]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100


In [44]:
# one hot encoding
downtown_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.010000,0.010000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.01,0.000000,0

In [47]:
num_top_venues = 50

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                            venue  freq
0                     Coffee Shop  0.06
1                 Thai Restaurant  0.04
2                      Steakhouse  0.04
3                             Bar  0.04
4                            Café  0.04
5                Sushi Restaurant  0.03
6                           Hotel  0.03
7                          Bakery  0.03
8                      Restaurant  0.03
9             American Restaurant  0.03
10                   Burger Joint  0.03
11                 Breakfast Spot  0.02
12                      Gastropub  0.02
13               Asian Restaurant  0.02
14                    Pizza Place  0.02
15                            Gym  0.02
16                 Cosmetics Shop  0.02
17                   Concert Hall  0.02
18                    Salad Place  0.02
19            Japanese Restaurant  0.02
20                     Food Court  0.01
21                   Dance Studio  0.01
22               Department Store  0.01
23     

Next step is to sort the top 50 venues by Neighbourhood and then use k means clustering to explore these venues. We want to see how common vegan restaurants are in these neighbourhoods.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(16, 51)

In [50]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


In [51]:
# set number of clusters
kclusters = 10

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 3, 1, 9, 5, 6, 0, 2, 2, 2, 4, 7, 8, 0, 3, 3], dtype=int32)

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge downtown_grouped with downtown_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood\n')

downtown_merged # check the last columns!

,Neighbourhood,Postcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,"Cabbagetown, St. James Town",M4X,Downtown Toronto,43.667967,-79.367675,9,Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Pub,Italian Restaurant,Caribbean Restaurant,Diner,Japanese Restaurant,Beer Store,Convenience Store,Bookstore,Sandwich Place,Breakfast Spot,Deli / Bodega,Chinese Restaurant,Pet Store,Liquor Store,Butcher,Snack Place,Playground,Market,Park,Pharmacy,Gift Shop,Jewelry Store,Gastropub,General Entertainment,Taiwanese Restaurant,Thai Restaurant,Indian Restaurant,Bank,Cosmetics Shop,Gay Bar,Creperie,Cupcake Shop,Dance Studio,Gaming Cafe,Department Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Fountain,Food Truck,Concert Hall,Dumpling Restaurant,Food & Drink Shop,Food Court,Dog Run
1,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Fast Food Restaurant,Gastropub,Gym,Mediterranean Restaurant,Men's Store,Burrito Place,Bubble Tea Shop,Pub,Café,American Restaurant,Adult Boutique,Health & Beauty Service,Bookstore,Pizza Place,Breakfast Spot,Diner,Park,Tea Room,Hobby Shop,Theater,Nightclub,Hotel,Caribbean Restaurant,Chinese Restaurant,Ice Cream Shop,Indian Restaurant,Dance Studio,Italian Restaurant,Theme Restaurant,Mexican Restaurant,Afghan Restaurant,Creperie,Juice Bar,Convenience Store,Wings Joint,Dog Run,Smoke Shop,Salon / Barbershop,General Entertainment,Seafood Restaurant,Sports Bar,Sculpture Garden,Video Game Store
2,"Harbourfront, Regent Park",M5A,Downtown Toronto,43.654260,-79.360636,8,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Brewery,Restaurant,Chocolate Shop,Performing Arts Venue,Cosmetics Shop,Dessert Shop,Beer Store,Electronics Store,Event Space,Farmers Market,Italian Restaurant,Ice Cream Shop,Hotel,French Restaurant,Historic Site,Health Food Store,Gym / Fitness Center,Shoe Store,Yoga Studio,Bank,Antique Shop,Spa,Beach,Fast Food Restaurant,Airport Service,Falafel Restaurant,Airport Terminal,Ethiopian Restaurant,American Restaurant,Dumpling Restaurant,Doner Restaurant,Donut Shop,Filipino Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Department Store,Aquarium,Airport Lounge,Airport Gate,Fish Market
3,"Ryerson, Garden District",M5B,Downtown Toronto,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Bar,Restaurant,Diner,Pizza Place,Bubble Tea Shop,Ramen Restaurant,Japanese Restaurant,Thai Restaurant,Theater,Lingerie Store,Chinese Restaurant,Office,Juice Bar,Department Store,Gym,Plaza,Gym / Fitness Center,Beer Bar,Lake,College Rec Center,Other Great Outdoors,Bookstore,Concert Hall,Hotel,Lounge,Hookah Bar,Music Venue,Burger Joint,Movie Theater,Pub,Comic Shop,Modern European Restaurant,Miscellaneou

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood\n'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Results: Analyze the 10 Clusters 

Ryerson, Garden District, Church and Wellesley Cluster 0

In [54]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
1,M4Y,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Fast Food Restaurant,Gastropub,Gym,Mediterranean Restaurant,Men's Store,Burrito Place,Bubble Tea Shop,Pub,Café,American Restaurant,Adult Boutique,Health & Beauty Service,Bookstore,Pizza Place,Breakfast Spot,Diner,Park,Tea Room,Hobby Shop,Theater,Nightclub,Hotel,Caribbean Restaurant,Chinese Restaurant,Ice Cream Shop,Indian Restaurant,Dance Studio,Italian Restaurant,Theme Restaurant,Mexican Restaurant,Afghan Restaurant,Creperie,Juice Bar,Convenience Store,Wings Joint,Dog Run,Smoke Shop,Salon / Barbershop,General Entertainment,Seafood Restaurant,Sports Bar,Sculpture Garden,Video Game Store
3,M5B,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Bar,Restaurant,Diner,Pizza Place,Bubble Tea Shop,Ramen Restaurant,Japanese Restaurant,Thai Restaurant,Theater,Lingerie Store,Chinese Restaurant,Office,Juice Bar,Department Store,Gym,Plaza,Gym / Fitness Center,Beer Bar,Lake,College Rec Center,Other Great Outdoors,Bookstore,Concert Hall,Hotel,Lounge,Hookah Bar,Music Venue,Burger Joint,Movie Theater,Pub,Comic Shop,Modern European Restaurant,Miscellaneous Shop,Mexican Restaurant,Caribbean Restaurant,Burrito Place,Fast Food Restaurant,Poutine Place,Food Court,Ethiopian Restaurant,Taco Place,Furniture / Home Store,Tea Room,Gastropub


CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara Cluster 1

In [55]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
13,M5V,1,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Plane,Boat or Ferry,Airport Food Court,Airport,Diner,Event Space,Comic Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Falafel Restaurant,Cupcake Shop,Concert Hall,Creperie,Cosmetics Shop,Convenience Store,Dance Studio,Filipino Restaurant,Farmers Market,Gastropub,Grocery Store,Greek Restaurant,Gourmet Shop,Gluten-free Restaurant,Gift Shop,German Restaurant,General Travel,General Entertainment,Gay Bar,Gaming Cafe,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Fountain,Food Truck


Design Exchange, Toronto Dominion Centre Cluster, Commerce Court, Victoria Hotel, First Canadian Place, Underground city,Adelaide, King, Richmond Cluster 2

In [56]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
7,M5H,2,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Bakery,Burger Joint,Restaurant,Sushi Restaurant,Hotel,American Restaurant,Asian Restaurant,Cosmetics Shop,Japanese Restaurant,Concert Hall,Salad Place,Gym,Breakfast Spot,Pizza Place,Gastropub,Office,Burrito Place,Building,New American Restaurant,Monument / Landmark,Brazilian Restaurant,Cocktail Bar,Opera House,Women's Store,Mediterranean Restaurant,Deli / Bodega,General Travel,Gluten-free Restaurant,Greek Restaurant,Electronics Store,Gym / Fitness Center,Department Store,Dance Studio,Lounge,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jazz Club,Plaza,Colombian Restaurant,Bookstore,Noodle House,Food Court,Salon / Barbershop,Smoke Shop
9,M5K,2,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Deli / Bodega,Burger Joint,Concert Hall,Steakhouse,Tea Room,Thai Restaurant,Pizza Place,Gym,Seafood Restaurant,Bakery,Beer Bar,Bar,Office,Bookstore,Breakfast Spot,Plaza,Pub,Noodle House,Japanese Restaurant,New American Restaurant,Museum,Ice Cream Shop,Hotel Bar,Gym / Fitness Center,Greek Restaurant,Gluten-free Restaurant,Fast Food Restaurant,General Travel,Food Court,Fried Chicken Joint,Basketball Stadium,French Restaurant,Art Gallery,Vegetarian / Vegan Restaurant,Speakeasy,Spa,Theater,Sporting Goods Shop,Train Station,Shopping Mall,Asian Restaurant,Sports Bar,Wine Bar
10,M5L,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery,Steakhouse,Salad Place,Bar,Gym,Fast Food Restaurant,Beer Bar,Thai Restaurant,Monument / Landmark,Pub,Bookstore,Poke Place,Creperie,Cupcake Shop,Breakfast Spot,Cocktail Bar,Hotel Bar,Fried Chicken Joint,Japanese Restaurant,Ice Cream Shop,Building,Burger Joint,Park,Department Store,Museum,Irish Pub,New American Restaurant,Pizza Place,French Restaurant,Greek Restaurant,Bank,Tailor Shop,Concert Hall,Art Gallery,Gym / Fitness Center,General Travel,Soup Place,Tea Room,Food Court,Shopping Mall,Food Truck
15,M5X,2,Coffee Shop,Café,Hotel,Burger Joint,Seafood Restaurant,American Restaurant,Bar,Deli / Bodega,Bakery,Steakhouse,Restaurant,Gastropub,Thai Restaurant,Tea Room,Asian Restaurant,Gym,Pizza Place,Japanese Restaurant,Beer Bar,Concert Hall,New American Restaurant,Burrito Place,Opera House,Building,Brazilian Restaurant,Museum,Noodle House,Monument / Landmark,Cocktail Bar,Mediterranean Restaurant,Lounge,Plaza,Cupcake Shop,Italian Restaurant,Ice Cream Shop,Hotel Bar,Department Store,Gym / Fitness Center,Greek Restaurant,Gluten-free Restaurant,General Travel,Fried Chicken Joint,Bookstore,Food Court,Art Gallery,Vegetarian / Vegan Restaurant,Wine Bar,Pub,Soup Place,Sushi Restaurant


St James, Stn A PO Boxes 25 The Esplanade Cluster, Berczy Park  3

In [57]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
4,M5C,3,Coffee Shop,Restaurant,Hotel,Café,Breakfast Spot,Italian Restaurant,Bakery,Cosmetics Shop,Gastropub,Seafood Restaurant,BBQ Joint,Cocktail Bar,Beer Bar,Clothing Store,Japanese Restaurant,Park,Diner,Gym,Farmers Market,American Restaurant,Church,Molecular Gastronomy Restaurant,New American Restaurant,French Restaurant,Performing Arts Venue,Cheese Shop,Camera Store,Movie Theater,Office,Middle Eastern Restaurant,Ice Cream Shop,Fountain,Food Truck,German Restaurant,Grocery Store,Electronics Store,Hostel,Indian Restaurant,Comfort Food Restaurant,Jazz Club,Department Store,Korean Restaurant,Latin American Restaurant,Pizza Place,Concert Hall,Creperie,Furniture / Home Store,Art Gallery,Theater,Accessories Store
5,M5E,3,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Pub,Steakhouse,Restaurant,Farmers Market,Seafood Restaurant,Beer Bar,Hotel,Basketball Stadium,Shopping Mall,Beach,Concert Hall,Cosmetics Shop,Bistro,Irish Pub,Italian Restaurant,Breakfast Spot,Liquor Store,Jazz Club,Park,Diner,Belgian Restaurant,Bagel Shop,BBQ Joint,Tea Room,Creperie,Gourmet Shop,Greek Restaurant,Vegetarian / Vegan Restaurant,Comfort Food Restaurant,Museum,French Restaurant,Thai Restaurant,Fountain,Tailor Shop,Art Gallery,Fish Market,Clothing Store,Dim Sum Restaurant,Dessert Shop,Dance Studio,Deli / Bodega,Cupcake Shop,Department Store,Discount Store,Women's Store
14,M5W,3,Coffee Shop,Restaurant,Café,Pub,Seafood Restaurant,Hotel,Cocktail Bar,Fast Food Restaurant,Cheese Shop,Beer Bar,Breakfast Spot,Bakery,Park,Farmers Market,Italian Restaurant,Lounge,Japanese Restaurant,Creperie,Cosmetics Shop,Art Gallery,Optical Shop,Comfort Food Restaurant,Bistro,Liquor Store,Concert Hall,Church,Bookstore,Poke Place,Hostel,Clothing Store,Deli / Bodega,Jazz Club,Diner,Movie Theater,Museum,Irish Pub,Molecular Gastronomy Restaurant,Belgian Restaurant,Salad Place,BBQ Joint,Gastropub,French Restaurant,Fountain,Vegetarian / Vegan Restaurant,Food Truck,Fish Market,Gourmet Shop,Thai Restaurant,Tea Room,American Restaurant


Harbord, University of Toronto Cluster 4

In [58]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
11,M5S,4,Café,Restaurant,Bookstore,Japanese Restaurant,Theater,Bar,Bakery,Sushi Restaurant,Sandwich Place,Pub,Comfort Food Restaurant,Beer Store,Beer Bar,Poutine Place,Italian Restaurant,Dessert Shop,Yoga Studio,College Arts Building,Chinese Restaurant,Nightclub,Gym,French Restaurant,Video Game Store,Noodle House,College Gym,Coffee Shop,Cupcake Shop,Discount Store,Diner,Dim Sum Restaurant,Dance Studio,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Department Store,Dog Run,Deli / Bodega,Ethiopian Restaurant,Doner Restaurant,Food Truck,German Restaurant,General Travel,General Entertainment,Gay Bar,Gastropub,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fountain


Central Bay Street Cluster 5


In [59]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 5, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
6,M5G,5,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Bar,Indian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Ice Cream Shop,Spa,Salad Place,Furniture / Home Store,Office,Park,Caribbean Restaurant,Pizza Place,Poke Place,Clothing Store,Comic Shop,Modern European Restaurant,Miscellaneous Shop,Ramen Restaurant,Korean Restaurant,Juice Bar,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Gym / Fitness Center,Falafel Restaurant,French Restaurant,Gastropub,Portuguese Restaurant,Yoga Studio,Smoothie Shop,Tea Room,Bakery,Vegetarian / Vegan Restaurant,Art Museum,Seafood Restaurant,Steakhouse,Sculpture Garden,American Restaurant,Wine Bar


Chinatown, Grange Park, Kensington Market Cluster 6

In [60]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 6, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
12,M5T,6,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Burger Joint,Farmers Market,Cocktail Bar,Dessert Shop,Caribbean Restaurant,Gaming Cafe,Record Shop,Ramen Restaurant,Japanese Restaurant,Belgian Restaurant,Poutine Place,Hospital,Pharmacy,Breakfast Spot,Brewery,Bubble Tea Shop,Pizza Place,Beer Bar,Park,Burrito Place,Organic Grocery,Hotel Bar,Ice Cream Shop,Noodle House,Cheese Shop,Jazz Club,Bistro,Sandwich Place,Comfort Food Restaurant,Wine Bar,Furniture / Home Store,Food Court,Thrift / Vintage Store,Liquor Store,Fish Market,Thai Restaurant,Tea Room,Filipino Restaurant,Korean Restaurant,Taco Place


Harbourfront East, Toronto Islands, Union Station Cluster 7

In [61]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 7, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
8,M5J,7,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Bakery,Pizza Place,Brewery,Restaurant,Baseball Stadium,History Museum,Park,Sporting Goods Shop,Sports Bar,Bar,Music Venue,Office,Chinese Restaurant,Lounge,Convenience Store,Japanese Restaurant,Basketball Stadium,Beer Bar,Plaza,Bistro,Indian Restaurant,Salad Place,Deli / Bodega,Performing Arts Venue,Ice Cream Shop,Monument / Landmark,Hotel Bar,Dance Studio,New American Restaurant,Bubble Tea Shop,Lake,Bank,Smoothie Shop,Wine Bar,Vegetarian / Vegan Restaurant,Train Station,Theater,Fast Food Restaurant,Tea Room,Event Space,Supermarket,Steakhouse,Art Gallery


Harbourfront, Regent Park Cluster 8

In [62]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 8, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
2,M5A,8,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Brewery,Restaurant,Chocolate Shop,Performing Arts Venue,Cosmetics Shop,Dessert Shop,Beer Store,Electronics Store,Event Space,Farmers Market,Italian Restaurant,Ice Cream Shop,Hotel,French Restaurant,Historic Site,Health Food Store,Gym / Fitness Center,Shoe Store,Yoga Studio,Bank,Antique Shop,Spa,Beach,Fast Food Restaurant,Airport Service,Falafel Restaurant,Airport Terminal,Ethiopian Restaurant,American Restaurant,Dumpling Restaurant,Doner Restaurant,Donut Shop,Filipino Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Department Store,Aquarium,Airport Lounge,Airport Gate,Fish Market


Cabbagetown, St. James Town Cluster 9

In [63]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 9, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue
0,M4X,9,Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Pub,Italian Restaurant,Caribbean Restaurant,Diner,Japanese Restaurant,Beer Store,Convenience Store,Bookstore,Sandwich Place,Breakfast Spot,Deli / Bodega,Chinese Restaurant,Pet Store,Liquor Store,Butcher,Snack Place,Playground,Market,Park,Pharmacy,Gift Shop,Jewelry Store,Gastropub,General Entertainment,Taiwanese Restaurant,Thai Restaurant,Indian Restaurant,Bank,Cosmetics Shop,Gay Bar,Creperie,Cupcake Shop,Dance Studio,Gaming Cafe,Department Store,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Fountain,Food Truck,Concert Hall,Dumpling Restaurant,Food & Drink Shop,Food Court,Dog Run


Results:
The downtown neighbourhoods were split into 10 clusters using K means clustering. Upon reviewing the top 50 venues of these clusters the following was determined:

Cluster 0- vegan restaurants were not amongst the top 50 venues.

Cluster 1 -  vegan restaurants were not amongst the top 50 venues.

Cluster 2- had vegan restaurants in their top 50 venues which included: 
Design Exchange/ Toronto Dominion Centre rated vegan restaurants as 41 most common venue/50 venues, and First Canadian Place, Underground city rated vegan restaurants as 46 most common venue/50 venues. 

Cluster 3- had vegan restaurants in their top 50 venues which included: 
Berczy Park rated vegan restaurants as 33 most common venue/50, and Stn A PO Boxes 25 The Esplanade rated vegan restaurants as 44 most common venue/50 

Cluster 4 -vegan restaurants were not amongst the top 50 venues. 

Cluster 5- Central Bay Street rated vegan restaurants as 44 most common venue/50. 

Cluster 6- rated vegan restaurants as 4 most common venue/50.

Cluster7- rated vegan restaurants as 42 most common venue/50.

Cluster 8- vegan restaurants were not amongst the top 50 venues.

Cluster 9- vegan restaurants were not amongst the top 50 venues.



Discussion:

One thing to mention, although restaurants in the top 50 may offer vegan options only those that were listed as "Vegetarian/Vegan Restaurants" were analyzed.

Upon analysis of the clusters, it was determined that 5/10 clusters had vegan restaurants rated in their top 50 most common venues, with the other 5 clusters having no vegan restaurants in their top 50. 2/10 clusters had more than one neighbourhood with vegan restuarants rated in the top 50. On average, most clusters that had vegan restaurants placed 33 or higher for most common venue, except cluster 6 which had highest rating as the 4th most common venue.

For future business owners, it is recommended to complete further analysis of of neighbourhhood needs and interested in vegan cuisine. If one were to open a vegan restaurant where there is little to no competition, clusters 0, 1, 4, 8, and 9 would be good options. For owners that want healthy competition and can depend on neighbourhood interested in vegan cuisine, all other clusters would be a great option. 


Conclusion

In conclusion, an anlysis of current vegan restaurant interest was conducted on the neighbourhoods in the downtown Toronto region. K means clustering was used to cluster the neighbourhoods into 10 clusters. Upon analysis, it was determined that 5/10 clusters had vegan restaurant ranked in their top 50 venues. For business owners, this is good news for those wishing to open up a vegan restaurant in the downtown Toronto region.